# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: I decided to use the LoRA technique, as advised, because I was most comfortable with what I was familiar with.
* Model: Again, decided to use GPT 2 because it was the most recommended in the instructions and FAQ
* Evaluation approach:I decided to use the imdb data set alongside the trainer API from Hugging Face, to ensure that any mistakes I made were not due to the usage of an unlearned evaluation method. 
* Fine-tuning dataset: I decided to fine tune a small portion of the imdb data set, so that the training would loop faster. I ended up choosing to train 500 examples from the imdb training split.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [7]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
import os
import torch

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [9]:
model = GPT2ForSequenceClassification.from_pretrained('gpt2')
model.config.pad_token_id = tokenizer.eos_token_id
model.gradient_checkpointing_enable()

checkpoint_dir = "/workspace/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoints = sorted(os.listdir(checkpoint_dir), reverse=True)
if len(checkpoints) > 3:
    os.remove(os.path.join(checkpoint_dir, checkpoints[-1]))  
checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_latest.pth")
torch.save(model.state_dict(), checkpoint_path)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

In [16]:
from datasets import load_dataset
import numpy as np
from transformers import TrainingArguments, Trainer

dataset=load_dataset("imdb")

split_dataset = dataset["train"].train_test_split(test_size=0.2, seed=23)
train_data = split_dataset["train"]
test_data = split_dataset["test"]

train_data.save_to_disk("data/imdb_custom_train")
test_data.save_to_disk("data/imdb_custom_test")


tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

training_args = TrainingArguments(
    output_dir="./results",
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

original_trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

original_results = original_trainer.evaluate()
print("Original Model:", original_results)

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Original Model: {'eval_loss': 0.7861188650131226, 'eval_accuracy': 0.5074, 'eval_runtime': 932.5803, 'eval_samples_per_second': 5.361, 'eval_steps_per_second': 0.67}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [17]:
from peft import LoraConfig, get_peft_model, TaskType
from peft import AutoPeftModelForSequenceClassification
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
from transformers import TrainingArguments
from datasets import load_from_disk

train_data = load_from_disk("data/imdb_custom_train")
test_data = load_from_disk("data/imdb_custom_test")


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_dir="./logs",             
    logging_steps=10,                 
    evaluation_strategy="epoch",     
    save_strategy="epoch",    
    
)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

peft_config = LoraConfig(
    r=8, 
    lora_alpha=16,
    target_modules=["c_attn"],  
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

peft_model = get_peft_model(model, peft_config)
model.config.pad_token_id = model.config.eos_token_id 

In [20]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.537000,0.489749,0.773800
2,0.567400,0.460634,0.792800
3,0.465200,0.462180,0.795800


TrainOutput(global_step=15000, training_loss=0.5435675119876862, metrics={'train_runtime': 31994.656, 'train_samples_per_second': 1.875, 'train_steps_per_second': 0.469, 'total_flos': 1966555791360000.0, 'train_loss': 0.5435675119876862, 'epoch': 3.0})

In [21]:
peft_model.save_pretrained("/tmp/peft-gpt2-imdb")

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [22]:
from transformers import GPT2Tokenizer
from datasets import load_dataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


dataset = load_dataset("imdb")

from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/tmp/results",
    per_device_eval_batch_size=4,
)

In [24]:
from peft import PeftModelForSequenceClassification

peft_model = PeftModelForSequenceClassification.from_pretrained(model,
    "/tmp/peft-gpt2-imdb"
)


tokenizer.pad_token = tokenizer.eos_token
peft_model.config.pad_token_id = tokenizer.eos_token_id

peft_trainer = Trainer(
    model=peft_model,
    args=training_args,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

fine_tuned_results = peft_trainer.evaluate()
print("Fine-Tuned Model:", fine_tuned_results)

Fine-Tuned Model: {'eval_loss': 0.4621804654598236, 'eval_accuracy': 0.7958, 'eval_runtime': 1603.8345, 'eval_samples_per_second': 3.118, 'eval_steps_per_second': 0.779}


In [25]:
print("Original Model:", original_results)
print("Fine-Tuned Model:", fine_tuned_results)

Original Model: {'eval_loss': 0.7861188650131226, 'eval_accuracy': 0.5074, 'eval_runtime': 932.5803, 'eval_samples_per_second': 5.361, 'eval_steps_per_second': 0.67}
Fine-Tuned Model: {'eval_loss': 0.4621804654598236, 'eval_accuracy': 0.7958, 'eval_runtime': 1603.8345, 'eval_samples_per_second': 3.118, 'eval_steps_per_second': 0.779}
